In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import gym
import numpy as np
from tensorboardX import SummaryWriter
from collections import namedtuple

In [ ]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [ ]:
class MlpNet(nn.Module):
    
    def __init__(self, obs_size, hidden_size, n_act):
        super(MlpNet, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_act)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])
Episode = namedtuple('Episode', field_names=['reward', 'steps'])

In [ ]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    flag1 = True

    while True:
        # convert obs to tensor
        obs_v = torch.Tensor([obs])

        act_prob_v = sm(net(obs_v))
        # converting act_prob to numpy
        act_prob = act_prob_v.data.numpy()[0]

        if flag1:
            print("act_prob: ", act_prob)
            flag1 = False

        action = np.random.choice(len(act_prob), p=act_prob)
        next_obs, reward, is_done, _ = env.step(action)

        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)

        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [ ]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    print("Reward Bound: ", reward_bound, "Reward Mean: ", reward_mean)

    train_obs = []
    train_act = []

    # batch: rwd & steps
    # steps: obs & act

    for reward, steps in batch:
        if reward < reward_bound:
            continue;
        train_obs.extend(map(lambda s: s.observation, steps))
        train_act.extend(map(lambda s: s.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)

    return train_obs_v, train_act_v, reward_bound, reward_mean    

In [ ]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    # #actions, here 2, left & right
    n_actions = env.action_space.n

    # network intialization
    net = MlpNet(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        
        # network training
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))

        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)

        if reward_m > 199:
            print("Solved!")
            break
    writer.close()